In [19]:
#1 Import the required libraries and modules that you would need.
import pandas as pd
import re
import numpy as np

In [20]:
#2 Read that data into Python and call the dataframe churnData.
churnData = pd.read_csv('Customer-Churn.csv')
churnData.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.5,No
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,Yes
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,Yes


In [21]:
#3 Check the datatypes of all the columns in the data. You would see that the column TotalCharges is object type. Convert this column into numeric type using pd.to_numeric function.

churnData.dtypes

def clean_empty(x):
    return re.sub(' ','0',x)

churnData['TotalCharges'] = churnData['TotalCharges'].apply(clean_empty)
churnData['TotalCharges'] = pd.to_numeric(churnData['TotalCharges'])

In [22]:
#4 Check for null values in the dataframe. Replace the null values.
pd.DataFrame(churnData.isna().sum())
#there are no nulls

,0
gender,0
SeniorCitizen,0
Partner,0
Dependents,0
tenure,0
PhoneService,0
OnlineSecurity,0
OnlineBackup,0
DeviceProtection,0
TechSupport,0


In [23]:
#5 Use the following features: tenure, SeniorCitizen, MonthlyCharges and TotalCharges:
#A Scale the features either by using normalizer or a standard scaler.

def function(x):
    if x == 'Yes':
        return re.sub('Yes','1',x)
    else:
        return re.sub('No','0',x)

churnData['Churn'] = churnData['Churn'].apply(function)
churnData['Churn'] = pd.to_numeric(churnData['Churn'])
    
X = churnData.loc[:,['tenure','SeniorCitizen','MonthlyCharges']]
y = churnData['Churn']

from sklearn.preprocessing import MinMaxScaler
# Normalizing data
transformer = MinMaxScaler().fit(X)
X_scaled = transformer.transform(X)
X = pd.DataFrame(X_scaled)
X.columns = ['tenure','SeniorCitizen','MonthlyCharges']


In [24]:
y

0       0
1       0
2       1
3       0
4       1
       ..
7038    0
7039    0
7040    0
7041    1
7042    0
Name: Churn, Length: 7043, dtype: int64

In [25]:
#B Split the data into a training set and a test set.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [26]:
#C Fit a logistic regression model on the training data.

from sklearn.linear_model import LogisticRegression
classification = LogisticRegression(random_state=0, solver='lbfgs',
                  multi_class='ovr').fit(X_train, y_train)

predictions = classification.predict(X_test)

#D Check the accuracy on the test data.
print(classification.score(X_test, y_test))

from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, predictions))

0.8041163946061036
[[959  77]
 [199 174]]


In [27]:
#4 Managing imbalance in the dataset
#A Check for the imbalance.
X = churnData.loc[:,['tenure','SeniorCitizen','MonthlyCharges']]
y = churnData['Churn']
imbalance_check = pd.DataFrame(y.value_counts())
print('imbalanced percentage:',round(imbalance_check['Churn'][0]/imbalance_check['Churn'].sum()*100,2),'%')

imbalanced percentage: 73.46 %


In [28]:
#B Use the resampling strategies used in class for upsampling and downsampling to create a balance between the two classes.
#Each time fit the model and see how the accuracy of the model is.

#DOWNSAMPLING:

from sklearn.utils import resample

category_0 = churnData[churnData['Churn'] == 0]
category_1 = churnData[churnData['Churn'] == 1]
category_0_undersampled = resample(category_0, replace=False, n_samples = len(category_1))

print(category_0_undersampled.shape)
print(category_1.shape)

data_downsampled = pd.concat([category_0_undersampled, category_1], axis=0)
data_downsampled['Churn'].value_counts()

X = data_downsampled.loc[:,['tenure','SeniorCitizen','MonthlyCharges']]
y = data_downsampled['Churn']

#building the model with downsample data:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from sklearn.linear_model import LogisticRegression
classification = LogisticRegression(random_state=0, solver='lbfgs',multi_class='ovr').fit(X_train, y_train)

predictions = classification.predict(X_test)

print(classification.score(X_test, y_test))

from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, predictions))

(1869, 16)
(1869, 16)
0.7312834224598931
[[270 109]
 [ 92 277]]


In [29]:
category_1

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,1
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,1
5,Female,0,No,No,8,Yes,No,No,Yes,No,Yes,Yes,Month-to-month,99.65,820.50,1
8,Female,0,Yes,No,28,Yes,No,No,Yes,Yes,Yes,Yes,Month-to-month,104.80,3046.05,1
13,Male,0,No,No,49,Yes,No,Yes,Yes,No,Yes,Yes,Month-to-month,103.70,5036.30,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7021,Male,0,No,No,12,Yes,No,No,No,Yes,Yes,No,One year,59.80,727.80,1
7026,Female,0,No,No,9,Yes,No,No,No,No,No,No,Month-to-month,44.20,403.35,1
7032,Male,1,No,No,1,Yes,No,No,No,No,No,No,Month-to-month,75.75,75.75,1
7034,Female,0,No,No,67,Yes,Yes,Yes,Yes,No,Yes,No,Month-to-month,102.95,6886.25,1


In [30]:
#UPSAMPLING

category_1_oversampled = resample(category_1, replace=True, n_samples = len(category_0))
print(category_0.shape)
print(category_1_oversampled.shape)
data_upsampled = pd.concat([category_0, category_1_oversampled], axis=0)
print(data_upsampled['Churn'].value_counts())

X = data_upsampled.loc[:,['tenure','SeniorCitizen','MonthlyCharges']]
y = data_upsampled['Churn']

#building the model with downsample data:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from sklearn.linear_model import LogisticRegression
classification = LogisticRegression(random_state=0, solver='lbfgs',multi_class='ovr').fit(X_train, y_train)

predictions = classification.predict(X_test)

print(classification.score(X_test, y_test))

from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, predictions))



(5174, 16)
(5174, 16)
1    5174
0    5174
Name: Churn, dtype: int64
0.7309178743961353
[[761 283]
 [274 752]]


In [31]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
model = DecisionTreeRegressor()

In [32]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=5,
                             min_samples_split=20,
                             min_samples_leaf =20)
clf.fit(X_train, y_train)
print(clf.score(X_train, y_train))
print(clf.score(X_test, y_test))

0.7564629137472819
0.7516908212560387


In [33]:
# For cross validation
from sklearn.model_selection import cross_val_score
clf = RandomForestClassifier(max_depth=5,
                             min_samples_split=20,
                             min_samples_leaf =20)
cross_val_scores = cross_val_score(clf, X_train, y_train, cv=10)
print(np.mean(cross_val_scores))

0.7510250366553926


In [34]:
cross_val_scores

array([0.76207729, 0.73550725, 0.76207729, 0.73671498, 0.76690821,
       0.73913043, 0.75966184, 0.76086957, 0.74244256, 0.74486094])